# Model 1

#### Install 

In [5]:
%pip install ultralytics
!pip install opencv-python-headless
!pip install labelme2yolo 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 716.2/716.2 kB 19.1 MB/s eta 0:00:0000:01
  Using cached opencv_python-4.9.0.80-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (62.2 MB)
  Using cached torch-2.2.1-cp311-cp311-manylinux1_x86_64.whl (755.6 MB)
  Using cached torchvision-0.17.1-cp311-cp311-manylinux1_x86_64.whl (6.9 MB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl (22 kB)
  Using cached thop-0.1.1.post2209072238-py3-none-any.whl (15 kB)
  Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none

#### Import Libraries 

In [1]:
import warnings
warnings.filterwarnings('ignore')

# GeoTiff Images
import rasterio 
from osgeo import gdal

# Visualisation
from matplotlib import pyplot as plt
import matplotlib.image as img
from matplotlib.pyplot import figure
from PIL import Image 

# Model Building
import ultralytics
from ultralytics import YOLO
import labelme2yolo 

# Azure 
from azure.storage.blob import BlobServiceClient 

# Others
import os
import shutil
import zipfile 
import re 
import subprocess
import shutil

%matplotlib inline 

In [2]:
# # Import images if not done already 
# !wget https://challenge.ey.com/api/v1/storage/admin-files/Post_Event_San_Juan.tif -O Post_Event_San_Juan.tif 
# !wget https://challenge.ey.com/api/v1/storage/admin-files/Pre_Event_San_Juan.tif -O Pre_Event_San_Juan.tif 

--2024-02-24 15:01:10--  https://challenge.ey.com/api/v1/storage/admin-files/Post_Event_San_Juan.tif
Resolving challenge.ey.com (challenge.ey.com)... 52.236.158.32
Connecting to challenge.ey.com (challenge.ey.com)|52.236.158.32|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1310494499 (1.2G) [application/octet-stream]
Saving to: ‘Post_Event_San_Juan.tif’

Post_Event_San_Juan 100%[===================>]   1.22G  44.6MB/s    in 27s     

2024-02-24 15:01:47 (47.1 MB/s) - ‘Post_Event_San_Juan.tif’ saved [1310494499/1310494499]

--2024-02-24 15:01:51--  https://challenge.ey.com/api/v1/storage/admin-files/Pre_Event_San_Juan.tif
Resolving challenge.ey.com (challenge.ey.com)... 52.236.158.32
Connecting to challenge.ey.com (challenge.ey.com)|52.236.158.32|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1016993213 (970M) [application/octet-stream]
Saving to: ‘Pre_Event_San_Juan.tif’

Pre_Event_San_Juan. 100%[===================>] 969.88M  68.1

In [13]:
import rasterio
from rasterio import windows
import os
import numpy as np
import random

def generate_random_tiles(src_path, dst_folder, random_windows, label):
    """
    Generate tiles from a GeoTIFF image using predefined windows and label them accordingly.
    """
    with rasterio.open(src_path) as src:
        out_meta = src.meta.copy()

        for count, window in enumerate(random_windows):
            img = src.read(window=window)
            if np.any(img):
                out_meta.update({
                    "driver": "GTiff",
                    "height": window.height,
                    "width": window.width,
                    "transform": rasterio.windows.transform(window, src.transform)
                })
                # Incorporate the label into the filename
                out_path = os.path.join(dst_folder, f"{label}_random_tile_{count}.tif")
                with rasterio.open(out_path, 'w', **out_meta) as out_dst:
                    out_dst.write(img)

def create_random_windows(width, height, tile_width, tile_height, max_tiles):
    """
    Generate a list of random windows for tile extraction.
    """
    random_windows = []
    for _ in range(max_tiles):
        rand_x = random.randint(0, width - tile_width)
        rand_y = random.randint(0, height - tile_height)
        window = windows.Window(rand_x, rand_y, tile_width, tile_height)
        random_windows.append(window)
    return random_windows

def main(pre_event_path, post_event_path, tile_width, tile_height, max_tiles=10):
    """
    Main function to generate random tiles from aligned pre-event and post-event images.
    """
    pre_event_dir = 'generated_data/Pre_Event_Tiles'
    post_event_dir = 'generated_data/Post_Event_Tiles'
    
    os.makedirs(pre_event_dir, exist_ok=True)
    os.makedirs(post_event_dir, exist_ok=True)

    # Open one of the images to determine dimensions for window generation
    with rasterio.open(pre_event_path) as src:
        width, height = src.width, src.height
    
    # Generate random windows
    random_windows = create_random_windows(width, height, tile_width, tile_height, max_tiles)

    # Generate tiles using the same set of random windows for both images, with appropriate labeling
    generate_random_tiles(pre_event_path, pre_event_dir, random_windows, "pre")
    generate_random_tiles(post_event_path, post_event_dir, random_windows, "post")

    print("Random tile generation completed for both pre-event and post-event images.")

if __name__ == "__main__": 

    pre_event_path = 'given/Pre_Event_San_Juan.tif'
    post_event_path = 'given/Post_Event_San_Juan.tif'
    tile_width, tile_height = 512, 512  # Adjust as needed

    main(pre_event_path, post_event_path, tile_width, tile_height)


Random tile generation completed for both pre-event and post-event images.


In [12]:
from PIL import Image
import os

def convert_tiff_to_jpeg(input_folder, output_folder):
    """
    Convert all TIFF images in the input_folder to JPEG format and save them in the output_folder.
    """
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)
    
    # Iterate through all TIFF files in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith('.tif') or filename.endswith('.tiff'):
            # Construct the full file path
            input_path = os.path.join(input_folder, filename)
            # Define the output file path with a .jpg extension
            output_path = os.path.join(output_folder, os.path.splitext(filename)[0] + '.jpg')
            
            # Open the TIFF image, convert to RGB (in case it's not), and save as JPEG
            with Image.open(input_path) as img:
                rgb_img = img.convert('RGB')
                rgb_img.save(output_path, 'JPEG')
    
    print(f"Conversion completed. JPEG images are saved in {output_folder}.")

# Repeat for post-event tiles if necessary
input_folder_post = 'generated_data/Post_Event_Tiles_Overlay'  # Adjust to your actual input folder path
output_folder_post = 'generated_data/Post_Event_Tiles_Overlay_JPEG'  # Adjust to your desired output folder path
convert_tiff_to_jpeg(input_folder_post, output_folder_post)


Conversion completed. JPEG images are saved in generated_data/Pre_Event_Tiles_Overlay_JPEG.
Conversion completed. JPEG images are saved in generated_data/Post_Event_Tiles_Overlay_JPEG.


In [14]:
import rasterio
from rasterio.windows import Window
import os

def crop_tiles_to_square(src_folder, dst_folder):
    """
    Crop all TIFF tiles in the source folder to square dimensions, maximizing the size of the image,
    and save them to the destination folder.
    """
    os.makedirs(dst_folder, exist_ok=True)  # Create destination folder if it doesn't exist

    for tile_filename in os.listdir(src_folder):
        if tile_filename.endswith('.tif') or tile_filename.endswith('.tiff'):
            src_path = os.path.join(src_folder, tile_filename)
            dst_path = os.path.join(dst_folder, tile_filename)

            with rasterio.open(src_path) as src:
                # Determine the minimum dimension to make the image square
                min_dim = min(src.width, src.height)
                # Calculate offsets for centering the crop
                off_x = (src.width - min_dim) // 2
                off_y = (src.height - min_dim) // 2

                # Define the window to crop the square area
                window = Window(off_x, off_y, min_dim, min_dim)
                window_transform = src.window_transform(window)

                # Read the data from the window
                data = src.read(window=window)

                # Update metadata for the output image
                out_meta = src.meta.copy()
                out_meta.update({
                    "driver": "GTiff",
                    "height": min_dim,
                    "width": min_dim,
                    "transform": window_transform
                })

                # Write the cropped square image
                with rasterio.open(dst_path, 'w', **out_meta) as dst:
                    dst.write(data)
    
    print("All tiles have been cropped to square dimensions and saved to the destination folder.")

# Example usage
src_folder = 'generated_data/Pre_Event_Tiles'  # Adjust to your actual source directory of tiles
dst_folder = 'generated_data/Pre_Event_Tiles_Square'  # Desired destination directory for square tiles
crop_tiles_to_square(src_folder, dst_folder)

# Repeat for post-event tiles if necessary
src_folder_post = 'generated_data/Post_Event_Tiles'  # Adjust to your actual source directory of tiles
dst_folder_post = 'generated_data/Post_Event_Tiles_Square'  # Desired destination directory for square tiles
crop_tiles_to_square(src_folder_post, dst_folder_post)


All tiles have been cropped to square dimensions and saved to the destination folder.
All tiles have been cropped to square dimensions and saved to the destination folder.
